<a href="https://colab.research.google.com/github/maxfloresv/FM849/blob/feat%2Fadd-class-11/20261/tutorials/11_lr_nlr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FM849 - Programación Científica para Proyectos de Inteligencia Artificial (IA)

👨‍🏫 **Profesores de cátedra**

- Máximo Flores Valenzuela ($\texttt{mflores@dcc.uchile.cl}$).
- Héctor Jiménez Orellana ($\texttt{hector.jimenezor@gmail.com}$).

## Regresión lineal

En la [clase 10](https://github.com/maxfloresv/FM849/blob/master/20261/10_lr_regularization.pdf) ya vimos los fundamentos de los modelos de regresión lineal: su formulación, sus supuestos, aplicaciones, etc. En este tutorial, veremos un ejemplo poniendo las "manos en la masa".

## Preámbulo

La siguiente es una configuración para cambiar el estilo de todos los gráficos. Sólo influye desde un punto de vista estético.

In [ ]:
import plotly.io as pio

pio.templates["cmu"] = pio.templates["plotly_white"]
pio.templates["cmu"]["layout"]["font"]["family"] = "CMU Sans Serif"
pio.templates["cmu"]["layout"]["font"]["size"] = 20
pio.templates["cmu"]["layout"]["title"] = {
  "x": 0.5,
  "xanchor": "center"
}
pio.templates["cmu"]["layout"].update(
  xaxis=dict(
    showgrid=False,
    zeroline=False,
    showline=True,
    linewidth=1,
    linecolor="black",
    ticks="outside",
    ticklen=8
  ),
  yaxis=dict(
    showgrid=False,
    zeroline=False,
    showline=True,
    linewidth=1,
    linecolor="black",
    ticks="outside",
    ticklen=8
  )
)

pio.templates["cmu"].layout.font.color = "black"
pio.templates["cmu"].layout.xaxis.color = "black"
pio.templates["cmu"].layout.yaxis.color = "black"

pio.templates.default = "cmu"

## 👨‍🦲 El caso del humano gigante

En este caso, buscaremos modelar la altura de las personas en función de su edad. Para ello, ocuparemos datos sobre el crecimiento de personas entre $2$ y $25$ años. Los datos se encuentran anotados en la siguiente celda de código.

In [ ]:
import numpy as np

edad = np.array([2, 5, 8, 10, 12, 14, 16, 18, 20, 25])
altura = np.array([87, 110, 127, 138, 150, 160, 170, 175, 176, 177])

Para explorar los datos, visualicemos cómo se ve la altura en función de la edad.

In [ ]:
import plotly.express as px

fig = px.scatter(
    x=edad,
    y=altura,
    title='Altura de personas en función de la edad'
  )
fig.update_layout(
    xaxis_title='Edad [años]',
    yaxis_title='Altura [cm]',
    width=800
  )
fig.show()

Ya podemos ver un pequeño problema que la regresión lineal no puede capturar: el cuerpo humano, biológicamente, tiene un límite de crecimiento. Para notar esto, reflexionen sobre la siguiente pregunta: **si se sigue la misma línea de crecimiento de la infancia, en casos normales, ¿cuánto deberíamos medir a los 60 años?**

Justamente esta noción de "línea de crecimiento" está alineada con el modelamiento de un fenómeno mediante una recta. Las rectas, al menos, si son crecientes, no vuelven a decrecer.

### Entrenando modelos de regresión

Consideremos un modelo de regresión lineal cualquiera, es decir, una variable objetivo $y$ y regresores $x_1, \dots, x_n$ con coeficientes $\beta_0, \beta_1, \dots, \beta_n$. Sabemos que este modelo se escribe como sigue:
$$
  y = \underbrace{\beta_0}_\text{intercepto} + \sum_{i=1}^n \underbrace{\beta_i \cdot x_i}_{\text{efecto de } x_i} + \underbrace{\varepsilon}_\text{residuo (error)}
$$

El objetivo es encontrar los parámetros $\beta_0$ y $\beta_1, \dots, \beta_n$ que reflejan el efecto que tiene cada variable predictora sobre la variable a predecir.

De las cátedras anteriores, ya sabemos que existen formas de calcular los coeficientes con una fórmula matemática, **sólo en el caso de regresión lineal**. Esto lo tiene implementado la clase `LinearRegression` de la librería `sklearn.linear_model`. Para entrenar (o ajustar) un modelo, sólo necesitamos pasarle $X$ (variables predictoras) e $y$ (variable a predecir).

En este caso, como la variable predictora es una sóla (`edad`), el ajuste se realiza instanciando a la clase (`lr = LinearRegression()`) y usando su método `fit(X, y)` (`lr.fit(X, y)`, equivalente a `lr.fit(edad.reshape(-1, 1), altura)` en nuestro caso).

> ❗ **Importante**: En vez de sólo `edad`, debemos usar `edad.reshape(-1, 1)`, que significa "toma la lista de edades y déjalas en una columna (el segundo `1`) en vez de una sóla fila". El `-1` se ocupa cuando no queremos especificar el número de filas, pero que se creen tantas como valores hayan (en este caso, una fila por cada edad que tenemos).

- Lo que hace la instanciación `lr = LinearRegression()` es crear una copia de la clase que ya implementa la regresión lineal para poder usar sus métodos.

- Lo que hace el `fit` es tomar los datos ($X, y$) para calcular los coeficientes $\beta_0, \beta_1, \dots, \beta_n$, que es el objetivo de la regresión lineal.

- Para revisar los coeficientes después de que el modelo fue ajustado, hay que usar `lr.intercept_` (intercepto $\beta_0$) y `lr.coef_` (coeficientes $\beta_1, \dots, \beta_n$; habrán tantos como variables predictoras hayan).

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(edad.reshape(-1, 1), altura)

print("Intercepto:", lr.intercept_)
print("Coeficientes asociados a variables:", lr.coef_)

Intercepto: 92.88169642857142
Coeficientes asociados a variables: [4.16294643]


Acá, sólo tenemos una variable, así que el modelo se puede escribir como $y = 92\text{,}882 + 4\text{,}163 \cdot x$, donde $y$ es la altura y $x$ la edad.

Con esto, confirmamos las sospechas que teníamos sobre el mal ajuste que tendría un modelo de regresión lineal, a menos que el dominio de $x$ (posibles valores que puede tomar) sea acotado. Notemos que para $x = 60$ años, tendremos una altura de $y = 342\text{,}662 \text{ [cm]}$, lo que no tiene sentido.

*Todos los valores de este ítem fueron aproximados a la milésima*.

## Realizando predicciones

Ya sabemos con qué datos fue entrenado nuestro modelo. Ahora, queremos hacer predicciones sobre datos nuevos. Para ello, se debe usar el método `predict(X)` (en nuestro caso, `lr.predict(nuevas_edades)`). Este método ya no recibe la variable objetivo $y$, porque es lo que queremos predecir a partir de las variables predictoras $X$.

La idea es que las nuevas edades no incluyan edades que el modelo ya conoce (las cuales están en `edad`), porque queremos probar qué tan bueno es su poder de generalización. Esta es la separación intuitiva entre el conjunto de entrenamiento y el de prueba.

Primero, recordemos qué edades teníamos.

In [ ]:
edad

array([ 2,  5,  8, 10, 12, 14, 16, 18, 20, 25])

Notemos que la edad $15$ no está en los datos de entrenamiento. Veamos qué predice el modelo para dicha edad. Como es una sóla, debemos usar la notación de tabla (o matriz) `[[15]]`, porque el método `predict` espera entradas en dos dimensiones, y una fila (p. ej., `[15]`) sólo tiene una dimensión.

In [ ]:
nuevas_edades = [[15], [17]]
pred = lr.predict(nuevas_edades)

pred

array([155.32589286, 163.65178571])

De todas formas, no pasa nada malo si ponemos edades que el modelo ya conoce del conjunto de entrenamiento. Solamente las métricas, si queremos evaluar la calidad del ajuste (que lo veremos más adelante), son menos confiables. Sin embargo, esto nos puede servir para entender comportamientos locales.

In [ ]:
nuevas_edades = [[14], [15], [16]]
pred = lr.predict(nuevas_edades)

pred

array([151.16294643, 155.32589286, 159.48883929])

Podemos ver que la predicción para $x = 15$ es la semisuma (o el promedio) entre las predicciones para las edades $x = 14$ y $x = 16$. Esto es así porque estamos modelando una recta, y las rectas tienen pendiente constante. El modelo entonces está entregando valores esperados, si dejamos de lado el contexto del problema.

## Graficando los residuos

La mayoría de las veces, para un análisis más completo, nos interesa visualizar los residuos del modelo, es decir, qué tan malo fue el ajuste para cada valor de edad del cual se posee información. En este análisis, se puede partir con el conjunto de entrenamiento y, posteriormente, revisar el conjunto de prueba (si están etiquetados, porque necesitamos la respuesta).

In [ ]:
import plotly.graph_objects as go

predicciones = lr.predict(edad.reshape(-1, 1))
residuos = altura - predicciones

fig = go.Figure(data=[go.Bar(x=edad, y=residuos)])
fig.add_shape(
    type="line",
    x0=min(edad), y0=0, x1=max(edad), y1=0,
    line=dict(color="Black", width=1, dash="dash")
)
fig.update_layout(
    title="Errores del modelo de regresión lineal",
    yaxis_title="Diferencia [cm]",
    xaxis_title="Edad [años]",
    width=800
)
fig.show()

Podemos ver que este modelo tiene residuos grandes en magnitud ($\pm 15$), lo que indica que no sobreajustó a los datos de entrenamiento. Además, podemos observar que los errores no tienen una forma aleatoria como sería esperado, sino que tienen una forma de "U" invertida. Esto indica que el fenómeno requiere una componente **no lineal** para ser modelado.

Nosotros conocemos cómo varían las alturas en función de la edad. Sabemos que hay un límite biológico a partir del cual el crecimiento se estanca. Esto, matemáticamente, lo podemos modelar con la siguiente función:

$$
h(t) = h_\text{máx} \cdot \left(1 - e^{-k t}\right)
$$

donde $h(t)$ es la altura en función de la edad ($t$), $h_\text{máx}$ es la altura máxima que se puede alcanzar, y $k$ la tasa de crecimiento (qué tan rápido alcanzamos esa altura máxima). Los parámetros a aprender son $h_\text{máx}$ y $k$.

Este modelo también tiene falencias. Por ejemplo, asume que todos los individuos tienen la misma altura máxima $h_\text{máx}$ y tasa de crecimiento $k$, lo que sabemos que es falso. Esta hipótesis la tenemos que imponer porque no tenemos más variables que estén prediciendo la altura, así que es netamente por falta de información.

Si tuviésemos el efecto del sexo en la altura (que sabemos que es una variable influyente), podríamos modelar $h(t)$ de una forma más precisa. Hay otras variables que también podríamos incluir, como el país de nacimiento, alimentación, etc.

Entendiendo que corresponde a una simplificación que busca mejorar la regresión lineal, implementémoslo en código. Para ello, necesitamos seguir los siguientes pasos:

1. Definir la función de modelamiento. En nuestro caso, ya sabemos que es $f(t) = h_\text{máx} \cdot \left(1 - e^{-kt}\right)$. Esto se guarda en la función `crecimiento` (pueden ponerle el nombre que ustedes quieran, pero deben ser consistentes).

2. Usar la función `curve_fit` de `scipy.optimize`, definiendo los siguientes hiperparámetros en orden: función de modelamiento (en nuestro caso, `crecimiento_simple`), variables predictoras ($X$; en nuestro caso, `edad`), variable objetivo ($y$; en nuestro caso, `altura`) y el paso inicial de la exploración, recordando que en los modelos de regresión no lineal se ocupan técnicas de aproximación (`p0=[h_max_inicial, k_inicial]`, en nuestro caso, `[180, 0.1]`). Este último hiperparámetro **debe tener el mismo orden** que fue definido en la función de modelamiento. En nuestro caso, va primero `h_max`, y después `k`. Si lo cambiáramos de orden, también deberíamos hacerlo en `p0`.

3. Extraer de la llamada a la función `curve_fit` lo que nos interesa. Esta función devuelve los parámetros óptimos (que llamamos `parametros`) y los errores de predicción (que llamamos `_`, porque no los usaremos: esto es una convención).


In [ ]:
from scipy.optimize import curve_fit

def crecimiento_simple(t, h_max, k):
  """
  Esta función modela el crecimiento tal y como lo hicimos arriba.
  """
  return h_max * (1 - np.exp(-k * t))

parametros, _ = curve_fit(crecimiento_simple, edad, altura, p0=[180, 0.1])

Este modelo ya está entrenado con los mismos datos que usamos para entrenar el modelo de regresión lineal, así que usaremos nuevamente $x=15$ para generar una predicción. En este caso, ya no es necesario usar una tabla de dos dimensiones (formato `[[15]]` que usamos en la regresión lineal), porque se trata de otra librería menos exigente con el formato.

In [ ]:
pred_no_lineal = crecimiento_simple(15, *parametros)
pred_no_lineal

np.float64(164.7429153869142)

Podemos notar que este valor se aleja casi $10 \text{ [cm]}$ de lo predicho por la regresión lineal. Grafiquemos los residuos.

In [ ]:
predicciones = crecimiento_simple(edad, *parametros)
residuos = altura - predicciones

fig = go.Figure(data=[go.Bar(x=edad, y=residuos)])
fig.add_shape(
    type="line",
    x0=min(edad), y0=0, x1=max(edad), y1=0,
    line=dict(color="Black", width=1, dash="dash")
)
fig.update_layout(
    title="Errores del modelo de regresión no lineal",
    yaxis_title="Diferencia [cm]",
    xaxis_title="Edad [años]",
    width=800
)
fig.show()

De todas formas, existe un problema. Los errores parecen tener un comportamiento más errático para edades mayores que $5$, pero al inicio tiene un error inaceptable (casi $30 \text{ [cm]}$). Esto pasa porque con la fórmula anterior, estamos asumiendo que las alturas parten de $0 \text{ [cm]}$, lo que no es así. Podemos probar añadiendo un parámetro $h_\text{base}$ que modele la altura basal de las personas, dejando la función como sigue:

$$
h(t) = h_\text{base} + \left(h_\text{máx} - h_\text{base}\right) \cdot \left(1 - e^{-kt}\right)
$$

Esto debería corregir el efecto de partir con una altura poco realista. $h_\text{base}$ es un nuevo parámetro que el modelo deberá aprender. Probemos con esta nueva formulación, indicando `p0=[50, 180, 0.1]`.

In [ ]:
def crecimiento_con_h_base(t, h_base, h_max, k):
  """
  Esta función modela el crecimiento con la fórmula actualizada.
  """
  return h_base + (h_max - h_base) * (1 - np.exp(-k * t))

parametros, _ = curve_fit(crecimiento_con_h_base, edad, altura, p0=[50, 180, 0.1])

Generemos el gráfico de los residuos nuevamente, para revisar cómo esto afectó a la solución.

In [ ]:
predicciones = crecimiento_con_h_base(edad, *parametros)
residuos = altura - predicciones

fig = go.Figure(data=[go.Bar(x=edad, y=residuos)])
fig.add_shape(
    type="line",
    x0=min(edad), y0=0, x1=max(edad), y1=0,
    line=dict(color="Black", width=1, dash="dash")
)
fig.update_layout(
    title="Errores del modelo de regresión no lineal con altura basal",
    yaxis_title="Diferencia [cm]",
    xaxis_title="Edad [años]",
    width=800
)
fig.show()

Este modelo es mucho mejor que los dos anteriores. Hemos logrado mejorarlo y con justificaciones claras.

Para finalizar, visualicemos en un gráfico un resumen del trabajo que hemos realizado.

In [ ]:
edades_a_comparar = np.linspace(0, 60, 100)

pred_lin = lr.predict(edades_a_comparar.reshape(-1, 1))

parametros, _ = curve_fit(
  crecimiento_simple,
  edad,
  altura,
  p0=[180, 0.1]
)
pred_no_lin_simple = crecimiento_simple(edades_a_comparar, *parametros)

parametros, _ = curve_fit(
  crecimiento_con_h_base,
  edad,
  altura,
  p0=[50, 180, 0.1]
)
pred_no_lin_con_hbase = crecimiento_con_h_base(edades_a_comparar, *parametros)

fig = go.Figure()

fig.add_trace(go.Scatter(
  x=edad,
  y=altura,
  mode='markers',
  name='Datos reales',
  marker=dict(color='black', size=8)
))

fig.add_trace(go.Scatter(
  x=edades_a_comparar,
  y=pred_lin,
  mode='lines',
  name='Lineal',
  line=dict(color='red', dash='dash')
))

fig.add_trace(go.Scatter(
  x=edades_a_comparar,
  y=pred_no_lin_simple,
  mode='lines',
  name='No lineal sin altura base',
  line=dict(color='orange', width=2)
))

fig.add_trace(go.Scatter(
  x=edades_a_comparar,
  y=pred_no_lin_con_hbase,
  mode='lines',
  name='No lineal con altura base',
  line=dict(color='green', width=3)
))

h_max_con_hbase = parametros[1]
fig.add_trace(go.Scatter(
  x=[min(edades_a_comparar), max(edades_a_comparar)],
  y=[h_max_con_hbase, h_max_con_hbase],
  mode='lines',
  name=f'Altura máxima con altura base',
  line=dict(color='gray', dash='dot')
))

fig.update_layout(
  title="Predicción de altura en el futuro",
  xaxis_title="Edad [años]",
  yaxis_title="Altura [cm]",
  width=800,
  height=600,
  legend=dict(
      x=0.01,
      y=0.99,
      xanchor='left',
      yanchor='top',
      bordercolor='Black',
      borderwidth=1,
      font=dict(size=16)
  )
)

fig.show()

Acá, se pueden confirmar varias afirmaciones que hicimos a lo largo del estudio.